In [1]:
# Celda 1: Setup
import mlflow
from pathlib import Path
import sys
from dotenv import load_dotenv
load_dotenv()

sys.path.insert(0, str(Path.cwd().parent))

from src.query_parser.parser import QueryParser

# MLflow
project_root = Path("/Users/santiagoamaya/Desktop/propAgent/real-estate-ai-finder")
mlflow.set_tracking_uri(f"file://{project_root}/mlruns")
mlflow.set_experiment("query_parser_experiments")

print("✅ Setup complete")

/opt/anaconda3/envs/realestate-ai/lib/python3.10/site-packages/mlflow/tracking/_tracking_service/utils.py:140: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, store_uri)
2025/11/26 19:45:42 INFO mlflow.tracking.fluent: Experiment with name 'query_parser_experiments' does not exist. Creating a new experiment.


✅ Setup complete


In [2]:
# Celda 2: Test queries de ejemplo
test_queries = [
    "Piso en Barcelona máximo 300k",
    "Local con entrada independiente Barcelona",
    "Vivienda luminosa 3 habitaciones Eixample",
    "Local comercial con dos ambientes, luz natural, máx 250k",
    "Busco casa en las afueras con jardín y parking"
]

parser = QueryParser(prompt_version="v1.0")

# Parse todas
with mlflow.start_run(run_name="baseline_v1.0"):
    results = []
    
    for query in test_queries:
        result = parser.parse(query, log_to_mlflow=False)
        results.append(result)
        
        print(f"\n📝 Query: {query}")
        print(f"   Direct: {result.direct_filters.dict(exclude_none=True)}")
        print(f"   Indirect: {result.indirect_filters.dict(exclude_none=True)}")
        print(f"   Confidence: {result.confidence:.2f}")
    
    # Log metrics agregadas
    avg_confidence = sum(r.confidence for r in results) / len(results)
    mlflow.log_metric("avg_confidence", avg_confidence)
    mlflow.log_param("prompt_version", "v1.0")
    mlflow.log_param("num_queries", len(test_queries))
    
print(f"\n✅ Average confidence: {avg_confidence:.2f}")


📝 Query: Piso en Barcelona máximo 300k
   Direct: {'location': 'barcelona-capital', 'property_type': 'piso', 'operation': 'comprar', 'max_price': 300000}
   Indirect: {'features': []}
   Confidence: 0.90


/var/folders/yv/mtqswh196p7drh_s54rdkgt80000gn/T/ipykernel_11686/3864360145.py:21: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  print(f"   Direct: {result.direct_filters.dict(exclude_none=True)}")
/var/folders/yv/mtqswh196p7drh_s54rdkgt80000gn/T/ipykernel_11686/3864360145.py:22: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  print(f"   Indirect: {result.indirect_filters.dict(exclude_none=True)}")



📝 Query: Local con entrada independiente Barcelona
   Direct: {'location': 'barcelona-capital', 'property_type': 'local', 'operation': 'comprar'}
   Indirect: {'entrance_type': 'independent', 'features': []}
   Confidence: 0.90

📝 Query: Vivienda luminosa 3 habitaciones Eixample
   Direct: {'location': 'barcelona-eixample', 'property_type': 'vivienda', 'min_rooms': 3}
   Indirect: {'natural_light': 'required', 'features': []}
   Confidence: 0.80

📝 Query: Local comercial con dos ambientes, luz natural, máx 250k
   Direct: {'property_type': 'local', 'operation': 'comprar', 'max_price': 250000}
   Indirect: {'natural_light': 'required', 'layout': 'dos ambientes', 'features': []}
   Confidence: 0.80

📝 Query: Busco casa en las afueras con jardín y parking
   Direct: {'property_type': 'casa', 'operation': 'comprar'}
   Indirect: {'features': ['jardín', 'parking']}
   Confidence: 0.70

✅ Average confidence: 0.82


In [3]:
# Celda 3: Análisis de edge cases
edge_cases = [
    "quiero algo en barcelona",  # Muy vago
    "precio hasta 500k 2 habitaciones", # Sin ubicación
    "local",  # Mínima información
]

for query in edge_cases:
    result = parser.parse(query)
    print(f"\n❓ Query: '{query}'")
    print(f"   Parsed: {result.dict(exclude_none=True)}")
    print(f"   Confidence: {result.confidence:.2f}")


❓ Query: 'quiero algo en barcelona'
   Parsed: {'original_query': 'quiero algo en barcelona', 'direct_filters': {'location': 'barcelona-capital'}, 'indirect_filters': {'features': []}, 'confidence': 0.3}
   Confidence: 0.30


/var/folders/yv/mtqswh196p7drh_s54rdkgt80000gn/T/ipykernel_11686/2789071524.py:11: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  print(f"   Parsed: {result.dict(exclude_none=True)}")



❓ Query: 'precio hasta 500k 2 habitaciones'
   Parsed: {'original_query': 'precio hasta 500k 2 habitaciones', 'direct_filters': {'operation': 'comprar', 'max_price': 500000, 'min_rooms': 2}, 'indirect_filters': {'features': []}, 'confidence': 0.8}
   Confidence: 0.80

❓ Query: 'local'
   Parsed: {'original_query': 'local', 'direct_filters': {'property_type': 'local'}, 'indirect_filters': {'features': []}, 'confidence': 0.7}
   Confidence: 0.70
